In [1]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 100)
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import numpy as np
import requests
from io import open as iopen

## 1. Street view api function

In [54]:
# global variable for API key

api = open(r"API_key_streetview.txt",)
api_KEY = api.read()


In [91]:
def streetview(dataframe,apikey,sizex,sizey,saveloc):
    properties = []
    base = "https://maps.googleapis.com/maps/api/streetview?"
    for index, row in dataframe.iterrows():
        uniqueid,address,zip_code = str(row[0]),str(row[4]),str(row[5])
        print(row[0]),str(row[4])
        prop = (uniqueid, address,zip_code)
        properties.append(prop)
#     status = []
#     unique_id = []
# add in a way to track the status codes
    for prop in properties:
        uniqueid,address,zip_code=prop
        size = "size={}x{}".format(sizex,sizey)
        location = "&location={}, Rochester,NY,{}".format(address,zip_code)
        key = "key={}".format(apikey)
        url = "{}{}&{}&{}".format(base,size,location,key)
        filename="{}/{}.png".format(saveloc,uniqueid)
        r = requests.get(url)
        print(r)
        print(filename)
        if r.status_code == requests.codes.ok:  
            with open(filename,'wb') as fd:
                fd.write(r.content)
        else:
            print(r.status_code)

## 2. read in deeds data

In [6]:
class_deeds = pd.read_csv("data/processed/deeds_12_21_classified.csv")

/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (5,6,9,85) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
investments = class_deeds.loc[class_deeds['investor_all']==1]

In [62]:
investments.head(2)

,Unnamed: 0,ID,swis_code,county_name,muni_name,muni_type,school_code,school_name,print_key,vlg_print_key,total_av,vlg_total_av,seller_last_name,seller_first_name,buyer_last_name,buyer_first_name,street_nbr_x,street_name_x,atty_last_name,atty_first_name,atty_phone,swis_county,book,page,deed_date,sale_date,sale_price,personal_prop,cod_usable,rar_usable,arms_length_flag,front,depth,total_sale_acres,nbr_of_parcels,prop_class_last_roll,prop_class_desc_last_roll,prop_class_at_sale,prop_class_desc_at_sale,grid_east,grid_north,zip5,zip4,buyer_zip5,buyer_zip4,buyer_last_name2,buyer_street_nbr,buyer_street_name,buyer_city,buyer_state,part_parcel_flag,condo_flag,ownership_code,contract_date,second_swis,new_const_flag,vlg_rar_usable,roll_yr,rps_update_flag,rps_update_date,cor_update_flag,oth_update_flag,last_fm_date,rollmatch_flag,form_recd,load_date,assessor_reviewed_flag,cond_business_sale,cond_buyer_is_seller,cond_company_sale,cond_deed_type,cond_estate,cond_govt_sale,cond_interest_conv,cond_multiple_swis,cond_other,cond_relative_sale,cond_signif_change,cond_memo,vlg_signif_change_flag,Unnamed: 78,sale_yr,geometry,seller_all_x,buyer_all_x_x,buyer_all_y,inv,occ,seller_all_y,buyer_all_x_y,deed_date_dt,street_nbr_y,street_name_y,elapsed,elapsed_days,multi_invest,investor_all,owner_occup
1,1.0,2,261400.0,Monroe,Rochester,2,261400.0,Rochester,106.420-0002-010.000,NaN,22000.0,0.0,WHELAN,MICHAEL,WHOLESALE REAL ESTATE OF,ROCHESTER LLC,59,COUNCIL ST 14605,NaN,NaN,NaN,26.0,11163.0,200.0,9/4/12,8/24/12,9000.0,0.0,N,N,Y,34.0,132.0,0.0,1.0,210.0,One Family Year-Round Residence,210.0,One Family Year-Round Residence,1412906.0,1157437.0,14605.0,NaN,14606.0,NaN,NaN,NaN,LYELL AV,ROCHESTER,NY,0.0,0.0,NaN,1/1/50,NaN,0.0,X,2012.0,1.0,11/21/12,0.0,0.0,12/6/12,1.0,1.0,11/16/12,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,12.0,POINT (1412906 1157437),MICHAEL WHELAN,ROCHESTER LLC WHOLESALE REAL ESTATE OF,True,1.0,0.0,NaN,NaN,NaN,NaN,NaN,999 days,NaN,NaN,1.0,0.0
2,2.0,3,261400.0,Monroe,Rochester,2,261400.0,Rochester,106.420-0002-010.000,NaN,22000.0,0.0,WHOLESALE REAL ESTATE OF,NaN,RAYA GRIA INC,NaN,59,COUNCIL ST 14605,NaN,NaN,NaN,26.0,11163.0,204.0,9/4/12,8/24/12,23000.0,0.0,Y,Y,Y,34.0,132.0,0.0,1.0,210.0,One Family Year-Round Residence,210.0,One Family Year-Round Residence,1412906.0,1157437.0,14605.0,NaN,14606.0,NaN,NaN,NaN,LYELL AV,ROCHESTER,NY,0.0,0.0,NaN,1/1/50,NaN,0.0,X,2012.0,1.0,11/21/12,0.0,0.0,12/6/12,1.0,1.0,11/16/12,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,12.0,POINT (1412906 1157437),nan WHOLESALE REAL ESTATE OF,nan RAYA GRIA INC,True,1.0,0.0,NaN,NaN,NaN,NaN,NaN,999 days,NaN,NaN,1.0,0.0


In [33]:
# just 2018
invest2018 = investments.loc[investments['sale_yr']==18.0]

In [61]:
invest2018 = invest2018.reset_index()
invest2018.head(3)

,level_0,index,Unnamed: 0,ID,swis_code,county_name,muni_name,muni_type,school_code,school_name,print_key,vlg_print_key,total_av,vlg_total_av,seller_last_name,seller_first_name,buyer_last_name,buyer_first_name,street_nbr_x,street_name_x,atty_last_name,atty_first_name,atty_phone,swis_county,book,page,deed_date,sale_date,sale_price,personal_prop,cod_usable,rar_usable,arms_length_flag,front,depth,total_sale_acres,nbr_of_parcels,prop_class_last_roll,prop_class_desc_last_roll,prop_class_at_sale,prop_class_desc_at_sale,grid_east,grid_north,zip5,zip4,buyer_zip5,buyer_zip4,buyer_last_name2,buyer_street_nbr,buyer_street_name,...,ownership_code,contract_date,second_swis,new_const_flag,vlg_rar_usable,roll_yr,rps_update_flag,rps_update_date,cor_update_flag,oth_update_flag,last_fm_date,rollmatch_flag,form_recd,load_date,assessor_reviewed_flag,cond_business_sale,cond_buyer_is_seller,cond_company_sale,cond_deed_type,cond_estate,cond_govt_sale,cond_interest_conv,cond_multiple_swis,cond_other,cond_relative_sale,cond_signif_change,cond_memo,vlg_signif_change_flag,Unnamed: 78,sale_yr,geometry,seller_all_x,buyer_all_x_x,buyer_all_y,inv,occ,seller_all_y,buyer_all_x_y,deed_date_dt,street_nbr_y,street_name_y,elapsed,elapsed_days,multi_invest,investor_all,owner_occup,address,zip,onlyaddr,address_full
0,0,11206,11200.0,100484,261400.0,Monroe,Rochester,2,261400.0,Rochester,105.330-0002-017.000,NaN,32000.0,0.0,LLC,BSMY,LAM,MICHAEL,24,ORIOLE ST 14613,MARCHIONI,JOHN,5852727870,26.0,12007.0,371.0,4/5/18,3/2/18,35000.0,0.0,Y,Y,Y,41.0,52.0,0.0,1.0,210.0,One Family Year-Round Residence,210.0,One Family Year-Round Residence,1399475.0,1158781.0,14614.0,NaN,11366.0,NaN,NaN,73-04,188TH STREET,...,NaN,1/4/18,NaN,0.0,X,2017.0,1.0,6/14/18,0.0,0.0,6/14/18,1.0,1.0,5/29/18,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,18.0,POINT (1399475 1158781),BSMY LLC,MICHAEL LAM,False,0.0,1.0,BSMY LLC,MICHAEL LAM,2018-04-05,24,ORIOLE ST 14613,15 days,15.0,1.0,1.0,0.0,24 ORIOLE ST,14613,24 ORIOLE ST,24 ORIOLE ST 14613
1,1,11208,11202.0,100490,261400.0,Monroe,Rochester,2,261400.0,Rochester,106.600-0002-073.000,NaN,21800.0,0.0,DIMAMBRO,PAOLO,ASIF KHAN MINHAS LLC,NaN,19,SIXTH ST 14605,PHETERSON,STEVEN J.,5853601203,26.0,12007.0,521.0,4/6/18,3/6/18,19500.0,0.0,Y,Y,Y,40.0,120.0,0.0,1.0,210.0,One Family Year-Round Residence,210.0,One Family Year-Round Residence,1415246.0,1155488.0,14605.0,NaN,14605.0,NaN,NaN,19,SIXTH STREET,...,NaN,2/2/18,NaN,0.0,X,2017.0,1.0,11/17/20,0.0,0.0,11/17/20,1.0,1.0,5/29/18,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,18.0,POINT (1415246 1155488),PAOLO DIMAMBRO,nan ASIF KHAN MINHAS LLC,True,1.0,0.0,NaN,NaN,NaN,NaN,NaN,999 days,NaN,NaN,1.0,0.0,19 SIXTH ST,14605,19 SIXTH ST,19 SIXTH ST 14605
2,2,11209,11203.0,100497,261400.0,Monroe,Rochester,2,261400.0,Rochester,091.670-0001-029.000,NaN,35000.0,0.0,SUOZZI N/K/A COOK,LISA M.,FARRELL,JOHN R.,1327,NORTON ST 14621,PHETERSON,STEVEN J.,5855465600,26.0,12007.0,655.0,4/6/18,3/12/18,45000.0,0.0,Y,Y,Y,36.0,99.0,0.0,1.0,210.0,One Family Year-Round Residence,210.0,One Family Year-Round Residence,1414028.0,1162818.0,14621.0,NaN,14621.0,NaN,NaN,1327,NORTON STREET,...,NaN,2/20/18,NaN,0.0,X,2017.0,1.0,6/14/18,0.0,0.0,6/14/18,1.0,1.0,5/29/18,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,18.0,POINT (1414028 1162818),LISA M. SUOZZI N/K/A COOK,JOHN R. FARRELL,False,0.0,1.0,LISA M. SUOZZI N/K/A COOK,JOHN R. FARRELL,2018-04-06,1327,NORTON ST 14621,999 days,NaN,1.0,1.0,0.0,1327 NORTON ST,14621,1327 NORTON ST,1327 NORTON ST 14621


In [76]:
invest2018['address_full'] = invest2018['street_nbr_x'] + " " + invest2018['street_name_x']

In [77]:
invest2018['address_full'].str[:-5]

0            24 ORIOLE ST           
1            19 SIXTH ST            
2          1327 NORTON ST           
3       0073-75 FERNCLIFFE DR       
4           497 MELVILLE ST         
                   ...              
773         658 BAY ST              
774     0019-21 ELMHURST ST         
775         252 DRIVING PK AV       
776         651 GARSON AV           
777    0865-869 CLIFFORD AV         
Name: address_full, Length: 778, dtype: object

In [78]:
invest2018['zip_code'] = invest2018['address_full'].str[-5:]

In [79]:
# separate out zip codes 
invest2018['address'] = invest2018['address_full'].str[:-5]

In [80]:
# manual correction
invest2018['address'][200] = '96 MELVILLE STREET'
invest2018['address'][373] = '362 AVIS AVENUE'

/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/darebrawley/anaconda3/envs/geo-2021/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [87]:
invest2018 = invest2018[['ID','print_key','buyer_all_x_x','seller_all_x','address','zip_code']]

In [88]:
test = invest2018.head(3)

In [93]:
streetview(invest2018,api_KEY,640,640,"exports/street_views/")

100484
100490
100497
100508
100512
100526
100527
100530
100540
100549
100550
100555
100562
100567
100581
100592
100601
100620
100622
100623
100627
100633
100634
100636
100637
100638
100639
100642
100643
100644
100652
100654
100661
100662
100667
100670
100671
100676
100686
100687
100696
100704
100718
100727
100735
100749
100751
100761
100773
100787
100796
100811
100829
100842
100852
100858
100875
100881
100885
100891
100903
100904
100926
100936
100939
100949
100952
100954
100959
100960
100975
100990
100991
100994
100999
101006
101023
101035
101037
101038
101040
101041
101042
101048
101049
101053
101062
101072
101083
101098
101099
101107
101139
101141
101142
101149
101158
101159
101162
101163
101167
101172
101182
101183
101189
101190
101191
101193
101200
101206
101229
101231
101234
101241
101258
101261
101265
101277
101286
101305
101320
101336
101337
101342
101349
101360
101363
101368
101371
101374
101377
101378
101382
101384
101392
101410
101412
101416
101432
101434
101445
101447
101451

<Response [200]>
exports/street_views//100858.png
<Response [200]>
exports/street_views//100875.png
<Response [200]>
exports/street_views//100881.png
<Response [200]>
exports/street_views//100885.png
<Response [200]>
exports/street_views//100891.png
<Response [200]>
exports/street_views//100903.png
<Response [200]>
exports/street_views//100904.png
<Response [200]>
exports/street_views//100926.png
<Response [200]>
exports/street_views//100936.png
<Response [200]>
exports/street_views//100939.png
<Response [200]>
exports/street_views//100949.png
<Response [200]>
exports/street_views//100952.png
<Response [200]>
exports/street_views//100954.png
<Response [200]>
exports/street_views//100959.png
<Response [200]>
exports/street_views//100960.png
<Response [200]>
exports/street_views//100975.png
<Response [200]>
exports/street_views//100990.png
<Response [200]>
exports/street_views//100991.png
<Response [200]>
exports/street_views//100994.png
<Response [200]>
exports/street_views//100999.png


<Response [200]>
exports/street_views//101881.png
<Response [200]>
exports/street_views//101887.png
<Response [200]>
exports/street_views//101894.png
<Response [200]>
exports/street_views//101896.png
<Response [200]>
exports/street_views//101910.png
<Response [200]>
exports/street_views//101914.png
<Response [200]>
exports/street_views//101916.png
<Response [200]>
exports/street_views//101924.png
<Response [200]>
exports/street_views//101925.png
<Response [200]>
exports/street_views//101926.png
<Response [200]>
exports/street_views//101938.png
<Response [200]>
exports/street_views//101940.png
<Response [200]>
exports/street_views//101967.png
<Response [200]>
exports/street_views//101985.png
<Response [200]>
exports/street_views//101993.png
<Response [200]>
exports/street_views//101994.png
<Response [200]>
exports/street_views//102014.png
<Response [200]>
exports/street_views//102023.png
<Response [200]>
exports/street_views//102025.png
<Response [200]>
exports/street_views//102029.png


<Response [200]>
exports/street_views//102917.png
<Response [200]>
exports/street_views//102926.png
<Response [200]>
exports/street_views//102937.png
<Response [200]>
exports/street_views//102938.png
<Response [200]>
exports/street_views//102940.png
<Response [200]>
exports/street_views//102951.png
<Response [200]>
exports/street_views//102952.png
<Response [200]>
exports/street_views//102953.png
<Response [200]>
exports/street_views//102957.png
<Response [200]>
exports/street_views//102979.png
<Response [200]>
exports/street_views//102983.png
<Response [200]>
exports/street_views//102990.png
<Response [200]>
exports/street_views//103014.png
<Response [200]>
exports/street_views//103016.png
<Response [200]>
exports/street_views//103025.png
<Response [200]>
exports/street_views//103043.png
<Response [200]>
exports/street_views//103048.png
<Response [200]>
exports/street_views//103050.png
<Response [200]>
exports/street_views//103064.png
<Response [200]>
exports/street_views//103065.png


<Response [200]>
exports/street_views//108583.png
<Response [200]>
exports/street_views//108593.png
<Response [200]>
exports/street_views//108595.png
<Response [200]>
exports/street_views//108623.png
<Response [200]>
exports/street_views//108625.png
<Response [200]>
exports/street_views//108627.png
<Response [200]>
exports/street_views//108640.png
<Response [200]>
exports/street_views//108642.png
<Response [200]>
exports/street_views//108651.png
<Response [200]>
exports/street_views//108658.png
<Response [200]>
exports/street_views//108675.png
<Response [200]>
exports/street_views//108687.png
<Response [200]>
exports/street_views//108709.png
<Response [200]>
exports/street_views//108725.png
<Response [200]>
exports/street_views//108736.png
<Response [200]>
exports/street_views//108740.png
<Response [200]>
exports/street_views//108753.png
<Response [200]>
exports/street_views//108763.png
<Response [200]>
exports/street_views//108764.png
<Response [200]>
exports/street_views//108769.png


<Response [200]>
exports/street_views//111998.png
<Response [200]>
exports/street_views//112008.png
<Response [200]>
exports/street_views//112009.png
<Response [200]>
exports/street_views//112010.png
<Response [200]>
exports/street_views//112011.png
<Response [200]>
exports/street_views//112012.png
<Response [200]>
exports/street_views//112023.png
<Response [200]>
exports/street_views//112024.png
<Response [200]>
exports/street_views//112033.png
<Response [200]>
exports/street_views//112034.png
<Response [200]>
exports/street_views//112044.png
<Response [200]>
exports/street_views//112053.png
<Response [200]>
exports/street_views//112077.png
<Response [200]>
exports/street_views//112086.png
<Response [200]>
exports/street_views//112090.png
<Response [200]>
exports/street_views//112101.png
<Response [200]>
exports/street_views//112113.png
<Response [200]>
exports/street_views//112116.png
<Response [200]>
exports/street_views//112117.png
<Response [200]>
exports/street_views//112118.png


In [94]:
invest2018.sort_values(by='address')

,ID,print_key,buyer_all_x_x,seller_all_x,address,zip_code
175,101607,120.550-0003-002.000,nan HORIZON REAL PROPERTY LLC,"nan CJB PROPERTIES, LLC",0001-3 SALINA ST,14611
597,109117,120.670-0001-057.000,nan TANIA'S KITCHEN TRUST,DAVID REYNOLDS,0002-4 ARNETT BLVD,14611
568,108779,121.740-0001-037.000,"nan FLIPPIN GREEKS, LLC",nan FREEMAN ASSOCIATES,0007-9 JAQUES ST,14620
82,101042,121.580-0002-021.000,ALLAN CHAPMAN,nan 1016 BOARDMAN STREET LLC,0010-12 BOARDMAN ST,14607
9,100549,106.290-0001-048.000,"nan CNR HOLDINGS, LLC",nan DOJA PROPERTIES NY LLC,0012-14 NIELSON ST,14621
...,...,...,...,...,...,...
423,103194,091.810-0003-024.000,MICHELE C. MULLEN,nan PRIME PER REALTY INC.,975 AVE D,14621
187,101653,105.560-0001-068.000,nan AWANA PROPERTIES INC.,nan RDM REAL ESTATE LLC,98 MC NAUGHTON ST,14606
344,102682,106.660-0002-050.001/0000,REALTY LLC STONO,"PROPERTIES, LLC WOODMOR",98 WOODWARD ST,14605
79,101038,122.550-0002-041.001/0000,ALMOND HILL PROPERTI LLC,GUBURNS LLC,987 HARVARD ST,14610
